<a href="https://colab.research.google.com/github/NapoliD/clasificacion/blob/master/XGBOOST_CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import pickle
import os
from sklearn.model_selection import train_test_split
#import lightgbm as lgb
import catboost as cb
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

os.chdir('')

# Create class: DataShell
class DataShell():
       
       
    # Initialize class with self and inputFile
    def __init__(self, inputFile):
        self.file = inputFile
                
    # Define generate_csv method, with self argument
    def generate_csv(self):
        self.data_as_csv = pd.read_csv(self.file)
        return self.data_as_csv
           
 

def organizar(X,y):
       # Splitting the dataset into the Training set and Test set
       X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)

       # Feature Scaling

       #sc = StandardScaler()
       
       #X_train = sc.fit_transform(X_train)
       #X_test = sc.transform(X_test)

       return X_train, X_test, y_train, y_test       


def calificar(X_train, y_train,X_test):
       
       # XGB
       classifier_cl = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0.2,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=5, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
       
       classifier_cl=classifier_cl.fit(X_train, y_train)
              
       y_pred_XGB = classifier_cl.predict(X_test)
       
       

       classifier=xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0.2,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=5, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
       
       classifier_xgb=classifier.fit(X_train ,y_train)
       
       y_pred_XGB_reg = classifier_xgb.predict(X_test)
       
       filename2 = 'XGB_Seg_Hogar_reg.sav'
       filename1 = 'XGB_Seg_Hogar_clas.sav'
       pickle.dump(classifier_xgb, open(filename2, 'wb'))
       pickle.dump(classifier_cl, open(filename1, 'wb'))
       
            
       # CAT
       
       classifier_CatBoost = cb.CatBoostClassifier(num_boost_round=50)
       regresor_CatBoost = cb.CatBoostRegressor(num_boost_round=50)
       classifier_CatBoost=classifier_CatBoost.fit(X_train, y_train)
       regresor_CatBoost=regresor_CatBoost.fit(X_train, y_train)
       
              
       filename3 = 'CatBoost_Seg_Hogar_Regre.sav'
       filename4 = 'CatBoost_Seg_Hogar_Clasi.sav'

       pickle.dump(regresor_CatBoost, open(filename3, 'wb'))
       pickle.dump(classifier_CatBoost, open(filename4, 'wb'))
       
       # Predicting the Test set results
       y_pred_CatBoost = classifier_CatBoost.predict(X_test)
              
       y_pred_CatBoost_regresor = regresor_CatBoost.predict(X_test)

       return y_pred_XGB,y_pred_XGB_reg,y_pred_CatBoost,y_pred_CatBoost_regresor
       
       
       
def calificacion(self):
              filename2 = 'XGB_Seg_Hogar_reg.sav'
              filename1 = 'XGB_Seg_Hogar_clas.sav'
              filename3 = 'CatBoost_Seg_Hogar_Regre.sav'
              filename4 = 'CatBoost_Seg_Hogar_Clasi.sav'
       
              XGB_PRESTAMO_reg = pickle.load(open(filename2, 'rb'))
              XGB_PRESTAMO_clas = pickle.load(open(filename1, 'rb'))
              CatBoost_ROBO_Regre = pickle.load(open(filename3, 'rb'))
              CatBoost_ROBO_Clasi = pickle.load(open(filename4, 'rb'))
              XGB_PRESTAMO_reg = XGB_PRESTAMO_reg.predict(self)
              XGB_PRESTAMO_clas = XGB_PRESTAMO_clas.predict(self)
              CatBoost_ROBO_Regre = CatBoost_ROBO_Regre.predict(self)
              CatBoost_ROBO_Clasi = CatBoost_ROBO_Clasi.predict(self)
              
              return  XGB_PRESTAMO_reg,XGB_PRESTAMO_clas,CatBoost_ROBO_Regre,CatBoost_ROBO_Clasi    
       
def Matriz_Confusion(y_test, y_pred_XGB):
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred_XGB)
    from sklearn.metrics import accuracy_score
    ac=accuracy_score(y_test, y_pred_XGB)
    from sklearn.metrics import classification_report
    cr=classification_report(y_test, y_pred_XGB)
    from sklearn.metrics import roc_auc_score
    rc=roc_auc_score(y_test, y_pred_XGB)
    sensibilidad=cm[1,1]/(cm[1,1]+cm[1,0])
    especificidad=cm[0,0]/(cm[0,0]+cm[0,1])
    
    
    return print(cm),print("Acurracy XGB:", ac),print(cr),print("AUC XGB:", rc),print("sensibilidad", sensibilidad),print("especificidad", especificidad)

